In [1]:
import pickle 
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import Functions as func
import var as var

Using plaidml.keras.backend backend.


In [2]:
#negatives 
df = pd.DataFrame(columns = ['path', 'label'])
negative = [f'../FinalImages/Negative/{i}' for i in os.listdir('../FinalImages/Negative')]
neg_labels = [0 for i in range(len(negative))]
positive = [f'../FinalImages/Positive/{i}' for i in os.listdir('../FinalImages/Positive')]
pos_labels = [1 for i in range(len(positive))]
concat_path = negative[:len(positive)+ len(positive)] + positive
concat_labels = neg_labels[:len(positive)+ len(positive)] + pos_labels


df.path = concat_path 
df.label = concat_labels 

X = df
y = df[['label']]

In [3]:
df.head(2)
print(df.label.value_counts())

0    144
1     72
Name: label, dtype: int64


In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify = X.label, train_size = .7, random_state = 10)

x_train.drop('label', axis = 1, inplace = True)
x_test.drop('label', axis = 1, inplace = True)

C:\Users\Heeeb\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
def get_img_array(img_paths, dim, img_type): 
    final_array = []
    for path in tqdm(img_paths): 
        img = func.get_image_value(path, dim, img_type)
        final_array.append(img)
    if img_type == 'grey':
        final_array = np.array(final_array)
        return final_array.reshape(final_array.shape[0], var.dimension[0],var.dimension[0],1)
    else: 
        return np.array(final_array)

new_x_train = get_img_array(x_train.path.values, var.mobilenet_dimension, img_type = var.img_type)
new_x_test = get_img_array(x_test.path.values, var.mobilenet_dimension, img_type = var.img_type)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 714.13it/s]


In [6]:
new_x_train.shape

(151, 224, 224, 3)

In [7]:
#save pickles

pickle.dump(new_x_train, open('../Pickles/mobile_net_x_train.p', 'wb'))
pickle.dump(y_train, open('../Pickles/mobile_net_y_train.p', 'wb'))
pickle.dump(new_x_test, open('../Pickles/mobile_net_x_test.p', 'wb'))
pickle.dump(y_test, open('../Pickles/mobile_net_y_test.p', 'wb'))

In [8]:
y_train.values
cv2.imshow('test', new_x_train[2])
cv2.waitKey(0)
cv2.destroyAllWindows()